# 利用osmnx与plot_map计算并可视化街道方向
本文的方向熵是指一个区域内所有街道的方向的混乱程度

In [1]:
import osmnx as ox
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import numpy as np
from plot_map import plot_map
import math

方向熵的计算公式为:
$$H = -\sum_{i=1}^nP(O_i)log_eP(O_i)$$

$P(O_i)$指方向在第i个方向街道的比例(这里将街道的方向分为了36类,将正北设置为0度,以10度为间隔创建36个区间,我们就认为它的方向就在这个区间)

In [ ]:
def reverse_bearing(x):
    return x + 180 if x < 180 else x - 180 

In [ ]:
def count_and_merge(n, bearings):
    # make twice as many bins as desired, then merge them in pairs
    # prevents bin-edge effects around common values like 0° and 90°
    n = n * 2
    bins = np.arange(n + 1) * 360 / n
    count, _ = np.histogram(np.array(bearings), bins=bins)
    
    # move the last bin to the front, so eg 0.01° and 359.99° will be binned together
    count = np.roll(count, 1)
    return count[::2] + count[1::2]

In [8]:
def calculate_entropy(count):
    count_p = count/count.sum()
    entropy = 0
    for i in count_p:
        entropy -= i*math.log(i)
    return entropy 

In [9]:
# function to draw a polar histogram for a set of edge bearings
def polar_plot(ax, bearings, n=36, title=''):

    bins = np.arange(n + 1) * 360 / n
    count = count_and_merge(n, bearings)
    _, division = np.histogram(bearings, bins=bins)
    frequency = count / count.sum()
    division = division[0:-1]
    width =  2 * np.pi / n

    ax.set_theta_zero_location('N')
    ax.set_theta_direction('clockwise')

    x = division * np.pi / 180
    bars = ax.bar(x, height=frequency, width=width, align='center', bottom=0, zorder=2,
                  color='#003366', edgecolor='k', linewidth=0.5, alpha=0.7)
    
    ax.set_ylim(top=frequency.max())
    
    title_font = {'family':'DejaVu Sans', 'size':24, 'weight':'bold'}
    xtick_font = {'family':'DejaVu Sans', 'size':10, 'weight':'bold', 'alpha':1.0, 'zorder':3}
    ytick_font = {'family':'DejaVu Sans', 'size': 9, 'weight':'bold', 'alpha':0.2, 'zorder':3}
    
    ax.set_title(title.upper(), y=1.05, fontdict=title_font)
    
    ax.set_yticks(np.linspace(0, max(ax.get_ylim()), 5))
    yticklabels = ['{:.2f}'.format(y) for y in ax.get_yticks()]
    yticklabels[0] = ''
    ax.set_yticklabels(labels=yticklabels, fontdict=ytick_font)
    
    xticklabels = ['N', '', 'E', '', 'S', '', 'W', '']
    ax.set_xticklabels(labels=xticklabels, fontdict=xtick_font)
    ax.tick_params(axis='x', which='major', pad=-2)

In [10]:
sh = gpd.read_file('/home/liu/Documents/pygeo-tutorial/shapefile/shanghai_shp/区县界_area.shp')

下面开始下载上海市各个区的路网数据,并计算每个区的路网的方向熵

In [ ]:
### 创造一个空的GeoDataFrame，用来储存我们即将下载的各个区的路网数据
sh_road = gpd.GeoDataFrame(columns = ['u', 'v', 'key', 'osmid', 'name', 'highway', 'oneway', 'length',
       'geometry', 'bridge', 'ref', 'lanes', 'maxspeed', 'access', 'tunnel',
       'junction','district'])

### 创造一个空的列表，用来存放各个区内各个街道的方向
sh_orientation_count = []

### 写一个循环，遍历上海的每个区，依次获取并处理路网数据
for i in sh.index:
    
    ### 利用osmnx，获取每个区的路网数据（可以通行小汽车的路网）
    G = ox.graph_from_polygon(sh.loc[i,'geometry'],network_type='drive')
    #sh.loc[i,'三岔路口比例'] = ox.stats.basic_stats(G)['streets_per_node_proportion'][3]
    #sh.loc[i,'十字路口比例'] = ox.stats.basic_stats(G)['streets_per_node_proportion'][4]
    
    ### 上一步获取的路网数据格式为networkx中的graph格式，这里我们将它转换成GeoDataFrame
    road_gdf = ox.graph_to_gdfs(G)[1]
    
    ### 并为路网赋上相应的行政区信息（属于哪个区）
    road_gdf['district'] = sh.loc[i,'NAME']
    ### 将每个区的路网添加至总的路网数据中
    sh_road = sh_road.append(road_gdf,ignore_index=True)
    
    ### 利用osmnx的add_edge_bearings函数为路网的边添加方向属性
    Gu = ox.add_edge_bearings(ox.get_undirected(G))
    
    ### 将边的方向属性都提取出来，存在一个Series中
    b = pd.Series([d['bearing'] for u, v, k, d in Gu.edges(keys=True, data=True)])
    
### 为边添加另一个方向的方向属性（+-180度）（因为路都是直线，如果从a端点到b端点与正北的夹角为30度，那么b端点到a端点与正北的夹角就是210度
    b = pd.concat([b, b.map(reverse_bearing)]).reset_index(drop=True).dropna()
    
    ### 将提取出来的方向属性添加到总的方向数据中
    sh_orientation_count.append(b)
    
    ### 计算每个区的街道的方向熵，并直接储存在上海的GeoDataFrame中
    sh.loc[i,'方向熵'] = calculate_entropy(count_and_merge(36,b))
    
    print('{}处理完毕'.format(sh.loc[i,'NAME']))

经过操作我们可以得到:
1. 上海的GeoDataFrame，储存了每个区的geometry和方向熵的具体数值
2. 上海的路网数据GeoDataFrame，储存了上海每个区的路网数据，有一个district字段专门来标记每条道路属于哪个区
3. 上海各个区的道路方向的集合（列表形式），储存了每个区内所有路网的方向

首先我们从上海的GeoDataFrame中提取每个区的方向熵，然后根据区的名称的字段，将其与上海路网数据的GeoDataFrame进行连接

这样我们可以为路网数据的GeoDataFrame添加一个方向熵的字段，代表每条道路所在区的路网的方向熵

In [ ]:
##提取街道名称与方向熵字段
sh_district_entropy = sh.loc[:,['NAME','方向熵']]

In [ ]:
##改一下名字,方便连接
sh_district_entropy.columns = ['district','方向熵']

In [ ]:
##根据district字段进行连接
sh_road = pd.merge(sh_road,sh_district_entropy,on='district',how = 'outer')

In [ ]:
sh_bounds = [120.79560279980944,30.623112899720564,122.03020320013769, 31.925845100091397]

In [ ]:
###开始绘图,先设置上海的行政区作为底图
base = sh.plot(figsize=(10,10),facecolor='none',edgecolor='black',lw=0.6)

###然后在底图上绘制路网地图,并以每条道路所在区的方向熵大小为路网赋予颜色(cmap操作)
sh_road.plot(ax=base,column='方向熵',lw=0.4,legend=True,legend_kwds={'shrink':0.7,'label':'Entropy'})

###标记方向熵大于75%分位数的区的名称
for i in sh.index:
    if sh.loc[i,'方向熵'] >= sh['方向熵'].quantile(0.75):
        plt.text(sh.loc[i,'geometry'].centroid.x,sh.loc[i,'geometry'].centroid.y,sh.loc[i,'NAME'],fontdict={'family':'Arial Unicode MS','size':10},horizontalalignment='center',verticalalignment='center')

### 调用plot_map添加osm的底图
plot_map(plt,sh_bounds,zoom=14,style=4)

### 关闭坐标轴，更好看
base.axis('off')

### 保存图片
plt.savefig('sh_entropy_plot.jpg',dpi=70,bbox_inches='tight')

plt.show()